In [ ]:
import requests
import pandas as pd

def extract_flights_info(flights, arr_icao, arr_airport):
	for l in flights:		
		# flight number
		try: flight_nr = l['number']
		except: flight_nr = None
		flights_dict['Flight_Number'].append(flight_nr)
		# departure airport
		try: dep_airport = l['departure']['airport']['name']
		except: dep_airport = None
		flights_dict['Departure_Airport'].append(dep_airport)
		# departure icao
		try: dep_icao = l['departure']['airport']['icao']
		except: dep_icao = None
		flights_dict['Departure_ICAO'].append(dep_icao)
		# arrival airport
		flights_dict['Arrival_Airport'].append(arr_airport)
		# arrival icao
		flights_dict['Arrival_ICAO'].append(arr_icao)
		# arrival terminal
		try: terminal = l['arrival']['terminal']
		except: terminal = None
		flights_dict['Arrival_Terminal'].append(terminal)
		# schedueled arrival time
		try: arr_time = l['arrival']['scheduledTimeLocal']
		except: arr_time = None
		flights_dict['Arrival_Time'].append(arr_time)
		# status of flight
		try: status = l['status']
		except: status= None
		flights_dict['Status'].append(status)
		# aircraft
		try: aircraft = l['aircraft']['model']
		except: aircraft = None
		flights_dict['Aircraft'].append(aircraft)
		# airline
		try: airline = l['airline']['name']
		except: airline = None
		flights_dict['Airline'].append(airline)

# make request to Aerodatabox API
def request_flights_data(API_key, arr_icao, from_local_time, to_local_time, arr_airport):
	headers = {
		"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
		"X-RapidAPI-Key": API_key
	}
	querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
	url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{arr_icao}/{from_local_time}/{to_local_time}"
	response = requests.request("GET", url, headers=headers, params=querystring)
	flights = response.json()['arrivals']
	extract_flights_info(flights, arr_icao, arr_airport)


API_key = "my-API-key"
from_local_time = "2022-04-05T08:00"
to_local_time = "2022-04-05T20:00"
# From which airport we want to extract arrival flights data
arr_icao = "EDDB"
arr_airport = "Berlin Brandenburg Airport"
flights_dict = {
	'Flight_Number':[],
	'Departure_Airport': [], 
	'Departure_ICAO': [],
	'Arrival_Airport':[],
	'Arrival_ICAO': [],
	'Arrival_Terminal': [],
	'Arrival_Time': [], 
	'Status': [], 
	'Aircraft': [],
	'Airline':[]
	}
request_flights_data(API_key, arr_icao, from_local_time, to_local_time, arr_airport)

flights_df = pd.DataFrame(flights_dict)	

In [ ]:
# before sending the table to MySQL, datatype of 'Arrival_Time' column should be changed to Datetime format
import numpy as np
schema="gans"
host="host-name-at-Amazon-Web-Service"
user="admin"
password="my-password-which-should-not-contain-@"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
(
flights_df
    .replace({np.nan},'unknown')
    .assign(Arrival_Time = lambda x: pd.to_datetime(x['Arrival_Time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))